In [ ]:
!pip install pysmt

In [ ]:
!pysmt-install --z3

In [43]:
from pysmt.shortcuts import *
from pysmt.typing import INT,BVType

Os problemas neste trabalho usam um “solver” SMT e são codificados em `pysmt`.


1. O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes a,b>0 ,  e devolve inteiros r,s,t tais que a\*s + b\*t = r  e r = gcd(a,b).
  Para além das variáveis r,s,t o código requer 3 variáveis adicionais r',s',t' que representam os valores de r,s,t no “próximo estado”.

```
INPUT  a, b
assume  a > 0 and b > 0
r, r', s, s', t, t' = a, b, 1, 0, 0, 1
while r' != 0:                                                          #0
  q = r div r'                                                          #1
  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'  #1
OUTPUT r, s, t                                                          #2
```

1. Construa um FOTS usando BitVector de tamanho n que descreva o comportamento deste programa: identifique as variáveis do modelo, o estado inicial e a relação de transição.
2. Considere estado de erro quando r=0 ou alguma das variáveis atinge o “overflow”. Prove que o programa nunca atinge o estado de erro.
3. Prove que a relação de Bézout, a\*s + b\*t = r é um invariante do algoritmo.

1.1) Construa um FOTS usando BitVector de tamanho n que descreva o comportamento deste programa: identifique as variáveis do modelo, o estado inicial e a relação de transição.

In [44]:
n=32

a=20
b=15

#mdc(a,b)

In [45]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)

    state['a'] = Symbol('a'+str(i),BVType(n))
    state['b'] = Symbol('b'+str(i),BVType(n))

    state['r'] = Symbol('r'+str(i),BVType(n))
    state['r_'] = Symbol('r_'+str(i),BVType(n))
    state['s'] = Symbol('s'+str(i),BVType(n))
    state['s_'] = Symbol('s_'+str(i),BVType(n))
    state['t'] = Symbol('t'+str(i),BVType(n))
    state['t_'] = Symbol('t_'+str(i),BVType(n))
    state['q'] = Symbol('q' +str(i),BVType(n))

    return state

In [46]:
def init(state):
    E = Equals(state['pc'],Int(0))

    A = Equals(state['a'],BV(a,n))
    B = Equals(state['b'],BV(b,n))

    R = Equals(state['r'],BV(a,n))
    R_ = Equals(state['r_'],BV(b,n))
    S = Equals(state['s'], BV(1,n))
    S_ = Equals(state['s_'],BV(0,n))
    T = Equals(state['t'],BV(0,n))
    T_ = Equals(state['t_'],BV(1,n))
    Q = Equals(state['q'],BV(0,n))
    return And(E,A,B,R,R_,S,S_,T,T_)

In [47]:
def trans(curr,prox):

    t01 = And(Equals(curr['pc'],Int(0)), Not(Equals(curr['r_'],BV(0,n))), Equals(prox['pc'],Int(1)),
              Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t02 = And(Equals(curr['pc'],Int(0)), Equals(curr['r_'],BV(0,n)), Equals(prox['pc'],Int(2)),
              Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t10 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(0)),
              Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(curr['q'], BVSDiv(curr['r'],curr['r_'])),
              Equals(prox['r'],curr['r_']), Equals(prox['r_'], BVSub(curr['r'],BVMul(curr['q'],curr['r_']))),
              Equals(prox['s'],curr['s_']), Equals(prox['s_'], BVSub(curr['s'],BVMul(curr['q'],curr['s_']))),
              Equals(prox['t'],curr['t_']), Equals(prox['t_'], BVSub(curr['t'],BVMul(curr['q'],curr['t_']))))

    t22 = And(Equals(curr['pc'],Int(2)), Equals(prox['pc'],Int(2)),
              Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    return Or(t01,t02,t10,t22)

In [48]:
def euclides(declare,init,trans,k):
    with Solver(name="z3") as solver:
      states = [declare(i) for i in range(k)]

      solver.add_assertion(init(states[0]))
      solver.add_assertion(And(trans(states[i],states[i+1]) for i in range(k-1)))


      if solver.solve():
        for i in range(k):
          print("Passo ",i)
          for j in states[i]:
            if j =='r':
              mdc=solver.get_value(states[i][j])
              print(j, "=", mdc,"<--")

            else:
              print(j, "=", solver.get_value(states[i][j]))
          print("--------")
        print(f'mdc({a},{b}) = {mdc}')

euclides(declare,init,trans,32)

Passo  0
pc = 0
a = 20_32
b = 15_32
r = 20_32 <--
r_ = 15_32
s = 1_32
s_ = 0_32
t = 0_32
t_ = 1_32
q = 1_32
--------
Passo  1
pc = 1
a = 20_32
b = 15_32
r = 20_32 <--
r_ = 15_32
s = 1_32
s_ = 0_32
t = 0_32
t_ = 1_32
q = 1_32
--------
Passo  2
pc = 0
a = 20_32
b = 15_32
r = 15_32 <--
r_ = 5_32
s = 0_32
s_ = 1_32
t = 1_32
t_ = 4294967295_32
q = 3_32
--------
Passo  3
pc = 1
a = 20_32
b = 15_32
r = 15_32 <--
r_ = 5_32
s = 0_32
s_ = 1_32
t = 1_32
t_ = 4294967295_32
q = 3_32
--------
Passo  4
pc = 0
a = 20_32
b = 15_32
r = 5_32 <--
r_ = 0_32
s = 1_32
s_ = 4294967293_32
t = 4294967295_32
t_ = 4_32
q = 0_32
--------
Passo  5
pc = 2
a = 20_32
b = 15_32
r = 5_32 <--
r_ = 0_32
s = 1_32
s_ = 4294967293_32
t = 4294967295_32
t_ = 4_32
q = 0_32
--------
Passo  6
pc = 2
a = 20_32
b = 15_32
r = 5_32 <--
r_ = 0_32
s = 1_32
s_ = 4294967293_32
t = 4294967295_32
t_ = 4_32
q = 0_32
--------
Passo  7
pc = 2
a = 20_32
b = 15_32
r = 5_32 <--
r_ = 0_32
s = 1_32
s_ = 4294967293_32
t = 4294967295_32
t_ = 4_32
q 

1.2) Considere estado de erro quando r=0 ou alguma das variáveis atinge o “overflow”. Prove que o programa nunca atinge o estado de erro.

Enquanto r' não é zero, o valor de r' sempre será menor que o valor anterior de r. Portanto, r' está em constante redução, mas nunca se torna zero até que o MDC seja encontrado.
Caso r seja igual a zero, o programa dá dois passos, pois entra no ciclo e realiza os calcúlos dentro do ciclo while, porem r terá o valor de r' e r' terá valor zero, assim saindo do ciclo no proximo passo.

Se tentarmos representar um número overflow, ou seja, que ocupe mais de 32 posições num vetor binário (BV), o pysmt apresentará um erro. Ou seja, para valores maiores que 2^n-1, será necessário atribuir um novo valor a n, de modo a possibilitar a representação do número em um bitvetor maior.

Portanto, desde que o tamanho dos BitVectors seja suficiente para representar os valores máximos esperados de a e b, o programa nunca atingirá o estado de erro, uma vez que o algoritmo de Euclides foi projetado para encontrar o MDC e respeitar os limites do tamanho dos BitVectors, e como a,b > 0, então r nunca obterá valor 0, pois este "herda" o valor do r' do ciclo anterior. Se no ciclo anterior tivemos r'=0, então não entra no cliclo pela condiçao do while.

1.3) Prove que a relação de Bézout, a\*s + b\*t = r é um invariante do algoritmo.

Queremos mostrar que a\*s + b\*t = r respeita as seguintes propriedades para ser invariante:


*   Inicialização: Antes de iniciar o ciclo a expressão deve ser verdadeira.
*   Preservação: A cada iteração do ciclo, a expressão deve ser verdadeira.

In [63]:
def inv(state):
    if Equals(BVAdd(BVMul(state['a'],state['s']),BVMul(state['b'],state['t'])), state['r']):
      return True
    else:                                 #a=20, b=15, 20*1+15*0=20
      return False

def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        with Solver(name="z3") as s:

            traco = [declare(i) for i in range(k)]

            # estado inicial
            s.add_assertion(init(traco[0]))

            # adicionar a função transição
            for i in range(k-1):
                s.add_assertion(trans(traco[i], traco[i+1]))

            # adicionar a negação do invariante
            s.add_assertion(Not(And(inv(traco[i]) for i in range(k-1))))

            if s.solve():
                for i in range(k):
                    print("Passo", i)
                    for v in traco[i]:
                        print(v, "=", s.get_value(traco[i][v]))
                    print('-------')
                print("A propriedade não é invariante")
                return

    print(f"O invariante mantém-se nos primeiros {K} passos")


bmc_always(declare,init,trans,inv,32)

AttributeError: ignored